<a href="https://colab.research.google.com/github/sarath-777/CodeGenerationFromGithubFiles/blob/main/DependencyGraph_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This gives the list of all groq api models available

In [13]:
import requests

from google.colab import userdata

GROQ_API_KEY = userdata.get('groqapi')

response = requests.get(
    "https://api.groq.com/openai/v1/models",
    headers={"Authorization": f"Bearer {GROQ_API_KEY}"}
)

print(response.json())  # Print available models


{'object': 'list', 'data': [{'id': 'qwen-qwq-32b', 'object': 'model', 'created': 1741214760, 'owned_by': 'Alibaba Cloud', 'active': True, 'context_window': 131072, 'public_apps': None}, {'id': 'distil-whisper-large-v3-en', 'object': 'model', 'created': 1693721698, 'owned_by': 'Hugging Face', 'active': True, 'context_window': 448, 'public_apps': None}, {'id': 'llama-3.2-90b-vision-preview', 'object': 'model', 'created': 1727226914, 'owned_by': 'Meta', 'active': True, 'context_window': 8192, 'public_apps': None}, {'id': 'allam-2-7b', 'object': 'model', 'created': 1737672203, 'owned_by': 'SDAIA', 'active': True, 'context_window': 4096, 'public_apps': None}, {'id': 'llama-3.3-70b-versatile', 'object': 'model', 'created': 1733447754, 'owned_by': 'Meta', 'active': True, 'context_window': 32768, 'public_apps': None}, {'id': 'llama3-70b-8192', 'object': 'model', 'created': 1693721698, 'owned_by': 'Meta', 'active': True, 'context_window': 8192, 'public_apps': None}, {'id': 'llama-guard-3-8b', '

In [1]:
pip install networkx requests PyGithub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 20.8 MB/s eta 0:00:00


In [3]:
pip install PyGithub networkx


2️⃣ fetch_repo.py → Clone & Extract Dependencies from GitHub
This script:
✅ Fetches the GitHub repository
✅ Extracts dependencies (imports, function calls, API routes, database changes)
✅ Builds a dependency graph

In [5]:
import os
import re
import json
import networkx as nx
from github import Github, InputGitTreeElement

# GitHub Authentication
from google.colab import userdata
GITHUB_TOKEN = userdata.get('githubReadAndWrite')
REPO_NAME = "sarath-777/demo_angular_springboot"  # Change this to your GitHub repo

# Initialize GitHub API client
g = Github(GITHUB_TOKEN)
repo = g.get_repo(REPO_NAME)

# Create a directed graph (dependencies have direction)
graph = nx.DiGraph()

def fetch_repo_files(repo):
    """Fetch all file paths in the repository."""
    contents = repo.get_contents("")
    files = []

    while contents:
        file_content = contents.pop(0)
        if file_content.type == "dir":
            contents.extend(repo.get_contents(file_content.path))
        else:
            files.append(file_content.path)

    return files

def extract_dependencies(file_content, file_path):
    """Extract dependencies from Angular & Spring Boot files."""
    dependencies = []

    # Detect Angular imports (TypeScript)
    if file_path.endswith(".ts"):
        matches = re.findall(r'import .* from ["\'](.*)["\'];', file_content)
        dependencies.extend(matches)

    # Detect Java imports
    elif file_path.endswith(".java"):
        matches = re.findall(r'import (.*);', file_content)
        dependencies.extend(matches)

    # Detect API calls
    api_matches = re.findall(r'httpClient\.(get|post|put|delete)\(["\'](.*?)["\']', file_content)
    dependencies.extend([api[1] for api in api_matches])

    return dependencies

def build_dependency_graph(repo):
    """Build a graph linking files based on dependencies."""
    files = fetch_repo_files(repo)

    for file_path in files:
        # Process only code files
        if not file_path.endswith((".ts", ".java", ".json", ".html")):
            continue  # Skip non-text files

        try:
            file_content = repo.get_contents(file_path).decoded_content.decode("utf-8")
        except UnicodeDecodeError:
            print(f"⚠️ Skipping binary/non-UTF-8 file: {file_path}")
            continue  # Skip files that can't be decoded as UTF-8

        graph.add_node(file_path)  # Add file as a node

        dependencies = extract_dependencies(file_content, file_path)

        for dep in dependencies:
            graph.add_edge(file_path, dep)  # Create dependency link

    return graph

# Build the graph
dependency_graph = build_dependency_graph(repo)

# Save graph locally in Colab
GRAPH_FILE = "dependency_graph.gml"
nx.write_gml(dependency_graph, GRAPH_FILE)
print(f"Dependency graph saved locally as '{GRAPH_FILE}'")

# Upload to GitHub in the `depGraphs/` folder
def upload_to_github(local_file, repo, github_path):
    """Upload the local file to GitHub repo."""
    with open(local_file, "r") as file:
        content = file.read()

    try:
        # Check if file already exists in GitHub
        contents = repo.get_contents(github_path)
        repo.update_file(github_path, "Updated dependency graph", content, contents.sha)
        print(f"✅ Updated '{github_path}' in GitHub repo.")
    except:
        repo.create_file(github_path, "Added dependency graph", content)
        print(f"✅ Uploaded '{github_path}' to GitHub repo.")

# Upload dependency graph to GitHub
upload_to_github(GRAPH_FILE, repo, "depGraphs/dependency_graph.gml")


Dependency graph saved locally as 'dependency_graph.gml'
✅ Uploaded 'depGraphs/dependency_graph.gml' to GitHub repo.


convert dependency graph to json format

In [6]:
import networkx as nx
import json

GRAPH_FILE = "dependency_graph.gml"
OUTPUT_JSON_FILE = "dependency_graph.json"

# Load the dependency graph
dependency_graph = nx.read_gml(GRAPH_FILE)

# Convert to JSON
graph_json = {
    "nodes": list(dependency_graph.nodes),
    "edges": list(dependency_graph.edges)
}

# Save as JSON
with open(OUTPUT_JSON_FILE, "w") as f:
    json.dump(graph_json, f, indent=4)

print(f"✅ Dependency graph saved as JSON: {OUTPUT_JSON_FILE}")


✅ Dependency graph saved as JSON: dependency_graph.json


code to send to groq api


In [7]:
import json
import requests


from google.colab import userdata

GROQ_API_KEY = userdata.get('groqapi')  # Replace with your Groq API key
GROQ_MODEL = "llama3-70b"  # Model name (choose an appropriate one)

# Load the dependency graph JSON
with open("dependency_graph.json", "r") as f:
    dependency_graph_json = json.load(f)

# New feature request (user input)
new_feature_request = """
I want to add a new feature that allows users to reset their password.
This requires:
- A new API endpoint in the backend
- A new frontend UI in Angular
- Updating existing authentication logic
"""

# Construct LLM Prompt
prompt = f"""
You are an AI assistant helping in software development.
Given the following dependency graph of a GitHub project and a new feature request,
identify which files will be affected and need modifications.

Dependency Graph:
{json.dumps(dependency_graph_json, indent=4)}

New Feature Request:
{new_feature_request}

Please analyze and list:
1. Files that need to be created or modified.
2. Changes required in each file.
"""

# Call Groq API
response = requests.post(
    "https://api.groq.com/v1/chat/completions",
    headers={"Authorization": f"Bearer {GROQ_API_KEY}", "Content-Type": "application/json"},
    json={
        "model": GROQ_MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.5
    }
)

# Process Response
llm_output = response.json()
suggestions = llm_output.get("choices", [{}])[0].get("message", {}).get("content", "")

print("📌 Suggested Changes:\n", suggestions)


📌 Suggested Changes:
 


the order of execution should be:
-llm fetches github repo files and structure
- llm analyzes the github repo files and structure
- llm analyzes the dependecy graph
- user gives new feature requeest
- llm  with the knowledge of github repo gives the code to the new feature request
- now based on the knowlege from the dependecy graph, the model identifies which files will be affected by the new feature suggestions and llm suggests to make modifications to those affected files inorder to avoid any errors or conflicts

Execution Steps:
1️⃣ LLM Fetches GitHub Repo Files & Structure
2️⃣ LLM Analyzes the GitHub Repo (Code, APIs, Components, DB, etc.)
3️⃣ LLM Analyzes the Dependency Graph
4️⃣ User Gives a New Feature Request
5️⃣ LLM Generates New Feature Code (Backend, Frontend, DB, API Updates)
6️⃣ LLM Identifies Affected Files & Suggests Modifications

1️⃣ & 2️⃣ LLM Fetches & Analyzes GitHub Repo

Modify the existing fetch_repo.py to send data directly to Groq API.

In [15]:
import json
import requests
from github import Github

# GitHub & Groq API credentials
from google.colab import userdata

GITHUB_TOKEN = userdata.get('githubReadAndWrite')
GROQ_API_KEY = userdata.get('groqapi')
REPO_NAME = "sarath-777/demo_angular_springboot"
GROQ_MODEL = "llama3-70b-8192"

# Initialize GitHub client
g = Github(GITHUB_TOKEN)
repo = g.get_repo(REPO_NAME)

def fetch_repo_structure(repo):
    """Fetch all files and their contents from the GitHub repo."""
    contents = repo.get_contents("")
    repo_data = {}

    while contents:
        file_content = contents.pop(0)
        if file_content.type == "dir":
            contents.extend(repo.get_contents(file_content.path))
        else:
            try:
                content = file_content.decoded_content.decode("utf-8")
            except UnicodeDecodeError:
                content = None  # Ignore non-text files
            repo_data[file_content.path] = content

    return repo_data

# Fetch repo data
repo_structure = fetch_repo_structure(repo)

# Send repo data to Groq API for analysis
prompt = f"""
Analyze the following GitHub repository structure.
Extract important files, their purposes, API endpoints, components, and database interactions.

Repository Data:
{json.dumps(repo_structure, indent=4)}
"""

response = requests.post(
    "https://api.groq.com/openai/v1/chat/completions",
    headers={"Authorization": f"Bearer {GROQ_API_KEY}", "Content-Type": "application/json"},
    json={
        "model": GROQ_MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3
    }
)

# Process Response

response_json = response.json()  # Get JSON response
print("🚀 Full API Response:\n", json.dumps(response_json, indent=4))  # Debug

if "choices" in response_json:
    llm_analysis = response_json["choices"][0]["message"]["content"]
    print("📌 LLM Analysis of GitHub Repo:\n", llm_analysis)
else:
    print("❌ Error: 'choices' key missing in API response!")

llm_analysis = response.json()["choices"][0]["message"]["content"]
print("📌 LLM Analysis of GitHub Repo:\n", llm_analysis)


🚀 Full API Response:
 {
    "error": {
        "message": "Request too large for model `llama3-70b-8192` in organization `org_01jn0nr2n4fmt855dgkax3cb6b` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 168647, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.",
        "type": "tokens",
        "code": "rate_limit_exceeded"
    }
}
❌ Error: 'choices' key missing in API response!


KeyError: 'choices'

🔥 Issue: Request Too Large for Groq API
Your request exceeds the 6,000 token per minute (TPM) limit on the Groq API. The error message states that your request used 168,647 tokens, which is too large.

In [16]:
import json
import requests
from github import Github
from google.colab import userdata

# GitHub & Groq API credentials
GITHUB_TOKEN = userdata.get('githubReadAndWrite')
GROQ_API_KEY = userdata.get('groqapi')
REPO_NAME = "sarath-777/demo_angular_springboot"
GROQ_MODEL = "llama3-70b-8192"

# Initialize GitHub client
g = Github(GITHUB_TOKEN)
repo = g.get_repo(REPO_NAME)

def fetch_repo_structure(repo, max_file_size=10000):
    """Fetch selected files (skip large/minor files) from the GitHub repo."""
    contents = repo.get_contents("")
    repo_data = {}

    while contents:
        file_content = contents.pop(0)
        if file_content.type == "dir":
            contents.extend(repo.get_contents(file_content.path))
        else:
            # Skip binary files and large files
            if file_content.size > max_file_size or not file_content.path.endswith((".ts", ".java", ".html", ".json")):
                continue
            try:
                content = file_content.decoded_content.decode("utf-8")
                # Summarize content if file is large
                if len(content) > 3000:
                    content = f"Summarized: {content[:1500]} ... {content[-1500:]}"
            except UnicodeDecodeError:
                content = None  # Ignore non-text files

            repo_data[file_content.path] = content

    return repo_data

# Fetch reduced repo data
repo_structure = fetch_repo_structure(repo)

# Send repo data to Groq API in **smaller batches**
chunk_size = 1000  # Number of characters per batch
repo_json = json.dumps(repo_structure, indent=4)

for i in range(0, len(repo_json), chunk_size):
    batch = repo_json[i:i+chunk_size]

    prompt = f"""
    Analyze the following GitHub repository files:
    {batch}
    Extract APIs, frontend components, and database details.
    """

    response = requests.post(
        "https://api.groq.com/openai/v1/chat/completions",
        headers={"Authorization": f"Bearer {GROQ_API_KEY}", "Content-Type": "application/json"},
        json={
            "model": GROQ_MODEL,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.3
        }
    )

    # Process Response
    response_json = response.json()

    if "choices" in response_json:
        llm_analysis = response_json["choices"][0]["message"]["content"]
        print(f"📌 LLM Analysis (Batch {i // chunk_size + 1}):\n", llm_analysis)
    else:
        print(f"❌ Error in batch {i // chunk_size + 1}: 'choices' key missing in API response!")


📌 LLM Analysis (Batch 1):
 Based on the provided `.eslintrc.json` file, here's what can be extracted:

**APIs:**

* None explicitly mentioned in this file. However, since it's an Angular project, it's likely that APIs will be defined in separate files, such as `environment.ts` or `api.service.ts`.

**Frontend Components:**

* The file mentions Angular-specific ESLint rules, which suggests that the frontend is built using Angular.
* The `directive-selector` and `component-selector` rules imply that the project uses Angular directives and components.
* The `prefix` property in these rules is set to `"app"`, which might indicate that the application prefix is "app" (e.g., `app-my-component`).

**Database Details:**

* None mentioned in this file. The `.eslintrc.json` file is primarily concerned with ESLint configuration for the frontend code, and database details are typically stored in separate configuration files or environment variables.

Other observations:

* The project uses TypeScr

3️⃣ LLM Analyzes Dependency Graph

In [17]:
# Load dependency graph
with open("dependency_graph.json", "r") as f:
    dependency_graph = json.load(f)

# Send graph to LLM
prompt = f"""
Analyze the following dependency graph of a GitHub repository.
This graph represents how different files depend on each other.

Dependency Graph:
{json.dumps(dependency_graph, indent=4)}

Extract key dependencies and their impact on major functionalities.
"""

response = requests.post(
    "https://api.groq.com/v1/chat/completions",
    headers={"Authorization": f"Bearer {GROQ_API_KEY}", "Content-Type": "application/json"},
    json={
        "model": GROQ_MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3
    }
)

# Process Response
graph_analysis = response.json()["choices"][0]["message"]["content"]
print("📌 LLM Analysis of Dependency Graph:\n", graph_analysis)


KeyError: 'choices'